In [ ]:
import imaplib
import email
from email.header import decode_header
import os

# --- Configuration ---
# !!! IMPORTANT: Replace with your actual credentials and server details !!!
# !!! Consider using environment variables or a config file for security !!!

MAILBOX = "INBOX"
DOWNLOAD_FOLDER = "email_attachments" # Folder to save attachments

# --- Helper Function to Decode Headers ---
def decode_subject(header):
    """Decodes email subject potentially encoded in multiple charsets."""
    decoded_parts = decode_header(header)
    subject = ""
    for part, charset in decoded_parts:
        if isinstance(part, bytes):
            subject += part.decode(charset or 'utf-8', errors='replace')
        else:
            subject += part
    return subject

# --- Main Logic ---
def fetch_emails_and_attachments():
    try:
        # Create the download folder if it doesn't exist
        if not os.path.exists(DOWNLOAD_FOLDER):
            os.makedirs(DOWNLOAD_FOLDER)

        # Connect to the server
        print(f"Connecting to {IMAP_SERVER}...")
        mail = imaplib.IMAP4_SSL(IMAP_SERVER)

        # Login
        print(f"Logging in as {EMAIL_ACCOUNT}...")
        mail.login(EMAIL_ACCOUNT, PASSWORD)

        # Select the mailbox
        status, messages = mail.select(MAILBOX)
        if status != 'OK':
            print(f"Error selecting mailbox {MAILBOX}: {status}")
            return

        print(f"Total messages in {MAILBOX}: {messages[0].decode()}")

        # Search for emails (e.g., 'ALL', 'UNSEEN', 'FROM "user@example.com"')
        # For this example, let's fetch all emails. For UNSEEN: 'UNSEEN'
        search_criteria = 'ALL'
        # search_criteria = 'UNSEEN'
        print(f"Searching for emails with criteria: {search_criteria}")
        status, email_ids = mail.search(None, search_criteria)

        if status != 'OK':
            print(f"Error searching for emails: {status}")
            mail.logout()
            return

        email_id_list = email_ids[0].split()
        print(f"Found {len(email_id_list)} email(s).")

        if not email_id_list:
            print("No emails found matching the criteria.")
            mail.logout()
            return

        # Fetch and process a limited number of emails to avoid overwhelming output
        # You can remove this limit or adjust it
        emails_to_process = email_id_list[-5:] # Get the latest 5 emails
        print(f"Processing the latest {len(emails_to_process)} email(s)...")


        for email_id in emails_to_process:
            print(f"\n--- Processing Email ID: {email_id.decode()} ---")
            # Fetch the email data (RFC822 means full email content)
            status, msg_data = mail.fetch(email_id, '(RFC822)')

            if status == 'OK':
                for response_part in msg_data:
                    if isinstance(response_part, tuple):
                        # Parse the email content
                        msg = email.message_from_bytes(response_part[1])

                        # Decode email subject
                        subject = decode_subject(msg["Subject"])
                        from_ = decode_subject(msg["From"])
                        to_ = decode_subject(msg["To"])
                        date_ = msg["Date"]

                        print(f"Subject: {subject}")
                        print(f"From: {from_}")
                        print(f"To: {to_}")
                        print(f"Date: {date_}")

                        # If the email is multipart
                        if msg.is_multipart():
                            # Iterate over email parts
                            for part in msg.walk():
                                # Extract content type of email
                                content_type = part.get_content_type()
                                content_disposition = str(part.get("Content-Disposition"))
                                try:
                                    body = part.get_payload(decode=True).decode(errors='replace')
                                except Exception as e:
                                    body = f"[Could not decode body part: {e}]"


                                if "attachment" in content_disposition:
                                    # Download attachment
                                    filename = part.get_filename()
                                    if filename:
                                        filename = decode_subject(filename) # Decode filename
                                        filepath = os.path.join(DOWNLOAD_FOLDER, filename)
                                        # Download attachment and save it
                                        with open(filepath, "wb") as f:
                                            f.write(part.get_payload(decode=True))
                                        print(f"Downloaded attachment: {filename}")
                                elif content_type == "text/plain" and "attachment" not in content_disposition:
                                    # Print text/plain part
                                    print("\nBody (Plain Text):")
                                    print(body)
                                elif content_type == "text/html" and "attachment" not in content_disposition:
                                    # You can save or process HTML body here if needed
                                    # For this example, we'll just indicate its presence
                                    print("\nBody (HTML): Present (content not printed to console)")
                                    # If you want to print it (can be long):
                                    # print(body)

                        else:
                            # Not a multipart email, extract the body directly
                            content_type = msg.get_content_type()
                            try:
                                body = msg.get_payload(decode=True).decode(errors='replace')
                            except Exception as e:
                                body = f"[Could not decode body: {e}]"

                            if content_type == "text/plain":
                                print("\nBody (Plain Text):")
                                print(body)
                            elif content_type == "text/html":
                                print("\nBody (HTML): Present (content not printed to console)")
                                # print(body) # Uncomment to print HTML body

                        # Optional: Mark email as read (seen)
                        # mail.store(email_id, '+FLAGS', '\\Seen')
                        # print(f"Marked email ID {email_id.decode()} as Seen.")

            else:
                print(f"Error fetching email ID {email_id.decode()}: {status}")

        # Logout
        mail.logout()
        print("\nLogged out.")

    except imaplib.IMAP4.error as e:
        print(f"IMAP Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    fetch_emails_and_attachments()

Connecting to imap.gmail.com...
Logging in as testingemailconnection6@gmail.com...
Total messages in INBOX: 5
Searching for emails with criteria: ALL
Found 5 email(s).
Processing the latest 5 email(s)...

--- Processing Email ID: 1 ---
Subject: gase
From: testing_email_connection <testingemailconnection6@gmail.com>
To: testing_email_connection <testingemailconnection6@gmail.com>
Date: Thu, 29 May 2025 10:59:16 +0200

Body (Plain Text):
games


Body (HTML): Present (content not printed to console)

--- Processing Email ID: 2 ---
Subject: Alerta de seguridad
From: Google <no-reply@accounts.google.com>
To: testingemailconnection6@gmail.com
Date: Thu, 29 May 2025 09:18:50 GMT

Body (Plain Text):
[image: Google]
Se ha añadido un número de teléfono para la verificación en dos pasos


testingemailconnection6@gmail.com
Los códigos para iniciar sesión en tu cuenta ahora se envían a un nuevo
número de teléfono. Si no has añadido ese número, alguien más podría estar
usando tu cuenta. Comprueba y 